# Project TOTO: Tornado Outbreak Threat Observations
ML Model: Developed by Michael Goodman

## Problem

## Assumption

## Data Engineering

### EDA

### Feature Engineering

## Model Approach

### Initial Model

### Parameter Tuning

### Model Selection

#### Justification

## Results and Insights

### Future Considerations

# Model Output